In [0]:
from pyspark.sql.functions import col, to_timestamp, year, month, hour, min, max
from pyspark.sql.utils import AnalysisException

In [0]:
%sql CREATE CATALOG IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS silver.spotify_eng;

In [0]:
from pyspark.sql.functions import (
    to_timestamp,
    col,
    year,
    month,
    hour
)

bronze_path = "bronze.spotify.spotify_analise"

df = (
    spark.table(bronze_path)
)

df = df.withColumn('ts', to_timestamp(col('ts')))
df = df.withColumn('year', year(col('ts')))
df = df.withColumn('month', month(col('ts')))
df = df.withColumn('hour', hour(col('ts')))
df = df.withColumn('minutes_played', col('ms_played') / 60000)
df = df.withColumn('hours_played', col('ms_played') / 3600000)

columns_to_check = [
    'master_metadata_track_name',
    'master_metadata_album_artist_name',
    'master_metadata_album_album_name',
    'spotify_track_uri'
]

df_nona = df.dropna(subset=columns_to_check)
df_stringnull = df_nona.filter(
    (col("master_metadata_track_name") != 'null') &
    (col("master_metadata_album_artist_name") != 'null') &
    (col("master_metadata_album_album_name") != 'null') &
    (col("spotify_track_uri") != 'null')
)

df_select = df_stringnull.select(
    "ts",
    "username",
    "platform",
    "ms_played",
    "master_metadata_track_name",
    "master_metadata_album_artist_name",
    "master_metadata_album_album_name",
    "spotify_track_uri",
    "reason_start",
    "reason_end",
    "shuffle",
    "skipped",
    "offline",
    "offline_timestamp",
    "incognito_mode",
    "year",
    "month",
    "hour",
    "minutes_played",
    "hours_played"
)

df_select.write.mode("overwrite").saveAsTable(
    "silver.spotify_eng.spotify_analise_2014_2024"
)

In [0]:
display(df_select)
#offline, skipped

In [0]:
display(
    df_select.select(
        min("ts").alias("min_ts"),
        max("ts").alias("max_ts")
    )
)

In [0]:
display(spark.sql("DESCRIBE TABLE silver.spotify_eng.spotify_analise_2014_2024"))